# SecretFlow is like React Hooks [alice]


In [ ]:
self_party: str = "alice"
peer_party: str = "bob"
self_ray_port: int = 32400
peer_ray_port: int = 32401
self_secretflow_port: int = 8080
peer_secretflow_port: int = 8081
self_spu_port: int = 8090
peer_spu_port: int = 8091

In [ ]:
from secretnote.compat.secretflow.device.driver import (
    SFConfigNetworked,
    SFClusterConfig,
    SFClusterParty,
)

secretflow_config = SFConfigNetworked(
    address=f"localhost:{self_ray_port}",
    cluster_config=SFClusterConfig(
        parties={
            self_party: SFClusterParty(address=f"127.0.0.1:{self_secretflow_port}"),
            peer_party: SFClusterParty(address=f"127.0.0.1:{peer_secretflow_port}"),
        },
        self_party=self_party,
    ),
)

In [ ]:
from secretnote.compat.spu import (
    SPUConfig,
    SPUClusterDef,
    SPUNode,
    SPURuntimeConfig,
    SPUProtocolKind,
    SPUFieldType,
)

spu_config = SPUConfig(
    cluster_def=SPUClusterDef(
        nodes=[
            SPUNode(party=self_party, address=f"127.0.0.1:{self_spu_port}"),
            SPUNode(party=peer_party, address=f"127.0.0.1:{peer_spu_port}"),
        ],
        runtime_config=SPURuntimeConfig(
            protocol=SPUProtocolKind.SEMI2K,
            field=SPUFieldType.FM128,
        ),
    )
)

In [ ]:
import secretflow

secretflow.init(**secretflow_config.dict())

In [ ]:
alice = secretflow.PYU(self_party)
bob = secretflow.PYU(peer_party)
spu = secretflow.SPU(**spu_config.dict())

In [ ]:
import jax.numpy as jnp


def multiply(x, y):
    return jnp.negative(jnp.multiply(x, y))

In [ ]:
import secretnote.functional as sfx

array = sfx.use_term(alice)(jnp.arange(10))
array = sfx.use_term(spu, array)()

multiplier = sfx.use_term(bob)(...)
multiplier = sfx.use_term(spu, multiplier)()

result = sfx.use_application(spu, array, multiplier)(multiply)

result_for_alice = sfx.use_term(alice, result)()
result_for_bob = sfx.use_term(bob, result)()

In [ ]:
secretflow.reveal(result_for_alice)